# **Homework**: DataTalksClub Data Engineering Zoomcamp dlt Workshop


In [2]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [3]:
# Import libraries
import dlt
import duckdb

# 1. Use a generator
**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

In [4]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

In [5]:
# Use generator 13 times
sum = 0
for i, sqrt_value in enumerate(generator, start=1):
    sum += sqrt_value
    print(f'Iteration number: {i}')
    print(f"square root value: {sqrt_value}")
    print(f"Cumulative sum: {sum}\n")

Iteration number: 1
square root value: 1.0
Cumulative sum: 1.0

Iteration number: 2
square root value: 1.4142135623730951
Cumulative sum: 2.414213562373095

Iteration number: 3
square root value: 1.7320508075688772
Cumulative sum: 4.146264369941973

Iteration number: 4
square root value: 2.0
Cumulative sum: 6.146264369941973

Iteration number: 5
square root value: 2.23606797749979
Cumulative sum: 8.382332347441762

Iteration number: 6
square root value: 2.449489742783178
Cumulative sum: 10.83182209022494

Iteration number: 7
square root value: 2.6457513110645907
Cumulative sum: 13.47757340128953

Iteration number: 8
square root value: 2.8284271247461903
Cumulative sum: 16.30600052603572

Iteration number: 9
square root value: 3.0
Cumulative sum: 19.30600052603572

Iteration number: 10
square root value: 3.1622776601683795
Cumulative sum: 22.4682781862041

Iteration number: 11
square root value: 3.3166247903554
Cumulative sum: 25.7849029765595

Iteration number: 12
square root value: 3.

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [6]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [7]:
# Append data from generators into empty list, and then append data to DuckDB table
data = []

# Append to empty list with both generators
for person in people_1():
  data.append(person)

for person in people_2():
  data.append(person)

# Create a pipeline
append_pipeline = dlt.pipeline(destination='duckdb', dataset_name='append')
info = append_pipeline.run(data,
                           table_name='people',
                           write_disposition='append')

# Create DuckDB connection
conn = duckdb.connect(f"{append_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{append_pipeline.dataset_name}'")

# Display tables in DuckDB
print('Loaded tables: ')
display(conn.sql("show tables"))

# Query people table
print("\n\n\n People table below:")
people = conn.sql("SELECT * FROM people").df()
display(people)

# Sum of people
print("\n\n\n Sum of ages:")
age = conn.sql("SELECT SUM(age) FROM people").df()
display(age)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 People table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707774427.8568456,+jj1SeHCg/uGsg,None
1,2,Person_2,27,City_A,1707774427.8568456,UrDEFE+ecb1HQw,None
2,3,Person_3,28,City_A,1707774427.8568456,QO3cnoUSnkbUOQ,None
3,4,Person_4,29,City_A,1707774427.8568456,+URW56xhErIqfQ,None
4,5,Person_5,30,City_A,1707774427.8568456,7XUAVWLbL7+JXg,None
5,3,Person_3,33,City_B,1707774427.8568456,rgkUyjIil5Qwnw,Job_3
6,4,Person_4,34,City_B,1707774427.8568456,p6r3yWWqjYfyXA,Job_4
7,5,Person_5,35,City_B,1707774427.8568456,9qLiIHJgFLbT/w,Job_5
8,6,Person_6,36,City_B,1707774427.8568456,TJULk59ogQpwQw,Job_6
9,7,Person_7,37,City_B,1707774427.8568456,BjlmTl7OUhnwEw,Job_7





 Sum of ages:


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [8]:
# to do: homework :)

# Append data using first generator
data = []
for person in people_1():
  data.append(person)

# Write data to DuckDB
# Create pipeline
pipeline1 = dlt.pipeline(destination='duckdb', dataset_name='merge')

# Run pipeline
info = pipeline1.run(data,
                     table_name='people',
                     primary_key="ID",
                     write_disposition='merge')

# Append to data with second generator
data = []
for person in people_2():
  data.append(person)

# Create pipeline
pipeline2 = dlt.pipeline(destination='duckdb', dataset_name='merge')

# Run pipeline
info = pipeline2.run(data,
                     table_name='people',
                     primary_key="ID",
                     write_disposition='merge')

In [9]:
## Show outcome of loading data
conn = duckdb.connect(f"{pipeline2.pipeline_name}.duckdb")

# Show tables
conn.sql(f"SET search_path = '{pipeline2.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# Query people table
print("\n\n\n People table below:")
rides = conn.sql("SELECT * FROM people").df()
display(rides)

# Sum of people
print("\n\n\n Sum of ages:")
age = conn.sql("SELECT SUM(age) FROM people").df()
display(age)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 People table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,2,Person_2,27,City_A,1707774429.0546515,fZLilR4WWpES/w,None
1,1,Person_1,26,City_A,1707774429.0546515,mPdOqgqpCGMNqg,None
2,5,Person_5,35,City_B,1707774430.2024136,PETZd9tQI0X5KQ,Job_5
3,7,Person_7,37,City_B,1707774430.2024136,fVjdVu0P9zf5aw,Job_7
4,4,Person_4,34,City_B,1707774430.2024136,idvZL7Dy4zeVxQ,Job_4
5,3,Person_3,33,City_B,1707774430.2024136,hc2gmFznVnPnXQ,Job_3
6,6,Person_6,36,City_B,1707774430.2024136,RLYtfnHRxAxeew,Job_6
7,8,Person_8,38,City_B,1707774430.2024136,lL+G0I7idFxW0A,Job_8





 Sum of ages:


,sum(age)
0,266.0
